In [10]:
"""
An Implementation of the method Neural Ordinary Differential 
Equation presented in: https://arxiv.org/abs/1806.07366


TODO: 
- implement a residual neural network
# - add the training loop
# - add the backpropagation

- implement a neural ODE


NOTES:
- residual structure doesn't make sense? inputs and outputs in the 
  residual block are being broadcasted as they don't have the same 
  dimensions. Also specifiying different depths has no effect on 
  the model predictions.

"""

import numpy as np
import pandas as pd

"""
Initialse the model parameters.
"""
def init_weights(layers, skips=[], scale=1.0, seed=0):
    
    # set the seed and create the weights
    rng = np.random.RandomState(seed)
    weights = [(scale * rng.randn(m, n), scale * rng.randn(n)) for m, n in zip(layers[:-1], layers[1:])]
    skip_weights = [None] * len(weights)    
    
    for skip in skips:
    
        # check the skips
        cond1, cond2 = (skip[1] - skip[0] > 1), (skip[0] > 0)
        cond3 = (skip[1] < len(weights))
        assert (cond1 & cond2 & cond3), "Invalid skip settings."
        
        # add skip
        n, m = weights[skip[0]][0].shape[0], weights[skip[1] - 1][0].shape[1]
        skip_weights[skip[1] - 1] = scale * rng.rand(n, m)
    
    return weights, skip_weights

"""
A basic residual neural network model set up so that 
skips are performed between layers of equal dimensions.
"""
class residual_NN:    
    def __init__(self, layers, skips=[], seed=0):
        
        # intialise the parameters
        self.weights, self.skip_weights = init_weights(layers, skips, seed)
        self.skip_start, self.skip_stop = zip(*skips)
        self.A = []
        
        # hyperparams
        self.lr = 1e-3
        self.lamba = 0.01
    
    """
    Get the forward prediction of shape (batch_size, state_dim)
    """
    def __call__(self, X):     
        
        A_log = [X]
        skip_start_log, skip_end_log = [None] * len(self.weights), [None] * len(self.weights)
        for idx, (w, b) in enumerate(self.weights):    
            
            # linear + activation
            Z = np.dot(X, w) + b
            A = np.tanh(Z) 
            
            # log hidden states
            A_log.append(A)
            
            # handle the skips layer
            if (idx + 1) in self.skip_start: skip_start_log[idx] = A
            if (idx + 1) in self.skip_stop: 
                start_idx = self.skip_start[self.skip_stop.index(idx + 1)]                   
                A_skip = np.dot(skip_start_log[start_idx - 1], self.skip_weights[idx])
                skip_end_log[idx] = A_skip
                A += A_skip    
            
            # move to next layer
            X = A
            
        # set intermediate states
        self.A = A_log[:-1] + [Z]
        self.A_skip_start, self.A_skip_end = skip_start_log, skip_end_log
        
        return Z   
    
    """
    Update the model weights.
    """
    def step(self, Y):
        
        skip_log = []
        for idx, (w, b) in reversed(list(enumerate(self.weights))):
            
            # compute the cost function
            N = Y.shape[0]
            if idx == (len(self.weights) - 1):                
                dz = np.sum(self.A[idx+1] - Y, axis=1, keepdims=True) 
                dw = (2/N) * np.dot(self.A[idx].T, dz) + (2/N) * (w * self.lamba)
                db = (2/N) * np.sum(dz, axis=0)                
                self.weights[idx] = (w - dw * self.lr, b - db * self.lr)                
                continue
            
            # update the hidden layers               
            dz_prev = dz
            dz = (1 - np.square(np.tanh(self.A[idx+1]))) * np.dot(dz, self.weights[idx+1][0].T) 
            
            # handle the skip
            if (idx + 2) in self.skip_start: dz += skip_log[-1]
            if (idx + 2) in self.skip_stop:                   
                dz_s = np.dot(dz_prev, self.skip_weights[idx + 1].T)                                
                dw_s = (1/N) * np.dot(self.A_skip_end[idx + 1].T, dz_s)                
                curr_ws = self.skip_weights[idx + 1]
                self.skip_weights[idx + 1] = curr_ws - (dw_s.T * self.lr)   
                skip_log.append(dz_s)                
            
            dw = (1/N) * np.dot(self.A[idx].T, dz) + (2/N) * (w * self.lamba)         
            db = (1/N) * np.sum(dz, axis=0)
            self.weights[idx] = (w - dw * self.lr, b - db * self.lr)            
            
"""
Simple Mean-Squared Error Loss
"""
def mse_loss(true, pred):   
    return np.mean(np.sum(np.square(true - pred), axis=1))

"""
Run the training loop for the residual model.
"""
def train_model(model, dataset, loss_func, epochs=10, batch_size=32, seed=0):
    
    rng = np.random.RandomState(seed)
    train, val = dataset[0], dataset[1]    
    for ep in range(epochs):
        
        # shuffle the datasets
        rng.shuffle(train)
        rng.shuffle(val)        
        
        losses = []
        iters = int(len(train) // batch_size) + 1 
        for it in range(iters):
            
            # get a batch of data            
            x_tr = train[it * batch_size: min((it + 1) * batch_size, len(train)), :-1]            
            y_tr = train[it * batch_size: min((it + 1) * batch_size, len(train)), -1]
                        
            # get the prediction            
            y_pred = model(x_tr)            
            
            # calculate the loss
            reg_loss = model.lamba * np.sum([np.sum(np.square(w)) for w, _ in model.weights])
            loss = loss_func(y_tr.reshape(-1, 1), y_pred) + reg_loss
                        
            # track losses
            losses.append(loss)
            
            # update the weights 
            model.step(y_tr.reshape(-1, 1))
            
        # get validation loss
        x_val, y_val = val[:, :-1], val[:, -1]            
        y_val_pred = model(x_val)
        reg_loss = model.lamba * np.sum([np.sum(np.square(w)) for w, _ in model.weights])
        val_loss = loss_func(y_val.reshape(-1, 1), y_val_pred) + reg_loss
        
        # display loss
        print('Ep: {} - Loss: {} - Val Loss:{} '.format(ep + 1, round(np.mean(losses), 3), round(val_loss, 3)))    


# load in the dataset
dataset = pd.read_csv("./Data/insurance_train.csv")
seed = 10

# convert columns to categorical
dataset["sex"] = dataset["sex"].astype('category')
dataset["region"] = dataset["region"].astype('category')
dataset["smoker"] = dataset["smoker"].astype('category')

# get the categorical columns
cat_columns = dataset.select_dtypes(['category']).columns
dataset[cat_columns] = dataset[cat_columns].apply(lambda x: x.cat.codes)

# convert to a numpy array and normalise
array = dataset.to_numpy()
mean, std = np.mean(array, axis=0), np.std(array, axis=0)
array = (array - mean) / std

# shuffle array and get split
rng = np.random.RandomState(seed)
rng.shuffle(array)
train_s, val_s, test_s = round(0.8 * len(array)), round(0.1 * len(array)), round(0.1 * len(array)) 
train, val, test = array[:train_s, :], array[train_s:(train_s+val_s), :], array[-test_s:, :]

# NOTE: ---------------------------------------------------------------

# layers: 60, 30, 20, 40
# (1, 3) -> output of 60 node layer appended to output of 20 node layer 

# ---------------------------------------------------------------------

# instantiate the model
x_dim, y_dim = train.shape[1] - 1, 1 
model = residual_NN(
    layers=[x_dim, 60, 60, 60, 60, y_dim],
    skips=[(1, 3)],
    seed=seed
)

# train the model
train_model(
    model=model,
    dataset=(train, val),
    loss_func=mse_loss,
    batch_size=128
)



Ep: 1 - Loss: 1.9773690827704805e+22 - Val Loss:2.5423118254614455e+22 
Ep: 2 - Loss: 2.542284845211306e+22 - Val Loss:2.54227312411891e+22 
Ep: 3 - Loss: 2.5422705134271795e+22 - Val Loss:2.5422648922554266e+22 
Ep: 4 - Loss: 2.5422621490621324e+22 - Val Loss:2.542256976873831e+22 
Ep: 5 - Loss: 2.5422540387155177e+22 - Val Loss:2.5422488033150364e+22 
Ep: 6 - Loss: 2.542245788457751e+22 - Val Loss:2.5422405106461456e+22 
Ep: 7 - Loss: 2.5422374435816607e+22 - Val Loss:2.5422320872015283e+22 
Ep: 8 - Loss: 2.542229020115614e+22 - Val Loss:2.5422236458356717e+22 
Ep: 9 - Loss: 2.5422205467195852e+22 - Val Loss:2.542215213156096e+22 
Ep: 10 - Loss: 2.5422120626965145e+22 - Val Loss:2.5422066621407434e+22 
